# Create a RAG with Responses API


![oai-responses](docs/oai-responses.png)

* [OpenAI Responses API](https://platform.openai.com/docs/api-reference/responses)
* [Azure OpenAI Responses API](https://learn.microsoft.com/en-us/azure/ai-foundry/openai/how-to/responses?tabs=python-key)
* [Vertex AI RAG Engine overview](https://cloud.google.com/vertex-ai/generative-ai/docs/rag-engine/rag-overview?_gl=1*1gh4852*_up*MQ..&gclid=CjwKCAjw2vTFBhAuEiwAFaScwheDO8BoxS9t35aQYzYpVChMuVZr8xP7S7bEUNvDPrXD7bD8gZOPKxoC_Z0QAvD_BwE&gclsrc=aw.ds)


https://platform.openai.com/docs/guides/retrieval

In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import os
from pathlib import Path
from dotenv import load_dotenv
from openai import OpenAI
from src import utils, conf

# Params

In [4]:
settings = conf.load(file="settings.yaml")
INDEX = settings.vdb_index
LLM = settings.llm_workhorse
RETRIEVE_K = 3

# Environment Variables

In [5]:
load_dotenv()

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

# Clients

In [6]:
client_openai = OpenAI(
    api_key=OPENAI_API_KEY
)

# Upload Data

In [7]:
utils.path_data_raw

WindowsPath('c:/Users/manuelalberto.romero/Documents/repos/dslabs/dslab-rag-e2e/data/raw')

In [8]:
# Identify every .pdf from data/raw/
pdf_files = [f for f in utils.path_data_raw.glob("*.pdf")]
print(f"{pdf_files=}")

pdf_files=[WindowsPath('c:/Users/manuelalberto.romero/Documents/repos/dslabs/dslab-rag-e2e/data/raw/3I_ATLAS, NAUKAS.pdf'), WindowsPath('c:/Users/manuelalberto.romero/Documents/repos/dslabs/dslab-rag-e2e/data/raw/Divulgacion-Planetaria-Althera.pdf'), WindowsPath('c:/Users/manuelalberto.romero/Documents/repos/dslabs/dslab-rag-e2e/data/raw/OMUAMUA. NAUKAS.pdf')]


In [9]:
file_path = utils.path_data_raw / "Divulgacion-Planetaria-Althera.pdf"
id_file = 'file-2CpCDMwstr55pejhrrFivr'  # add file id once created from cell output print on the first run

lst_oai_files = [x.id for x in client_openai.files.list()]

if id_file not in lst_oai_files:
    response_file = client_openai.files.create(
        file=open(file_path, 'rb'),
        purpose="assistants",
        expires_after={
            "anchor": "created_at",
            "seconds": 2592000   # 30d
        })
    print(f"Added file, and set as id_file: {response_file.id=}")
    id_file = response_file.id
else:
    print(f"File: {id_file} alredy added")

File: file-2CpCDMwstr55pejhrrFivr alredy added


# Index

In [10]:
id_vdb = "vs_698f014ef4648191bbcab8191d249673"   # add vector store id once created from cell output print on the first run

lst_oai_vdb =[x.id for x in client_openai.vector_stores.list()]

if id_vdb not in lst_oai_vdb:
    response_attach = client_openai.vector_stores.create(
        name=INDEX,
        file_ids=[response_file.id],
        expires_after={
            "anchor": "last_active_at",
            "days": 30
            }
    )
    print(f"Created VDB and set as id_vdb: {response_attach.id=}")
    id_vdb = response_attach.id
else:
    print(f"VDB: {id_vdb} alredy exists")



VDB: vs_698f014ef4648191bbcab8191d249673 alredy exists


In [11]:
# for expired vdbs, list them and them delete
# client_openai.vector_stores.delete('vs_68bd8ba217fc8191813ca8ee1a846fc3')

In [11]:
# max_chunk_size_tokens of 800 and chunk_overlap_tokens of 400.
vector_store_files = client_openai.vector_stores.files.list(
  vector_store_id=id_vdb
)

lst_indexed_files = [x.id for x in vector_store_files.data]
if id_file not in lst_indexed_files:
    client_openai.vector_stores.files.create(  # ocr, chunk, embeddings and index
        vector_store_id=id_vdb,
        file_id=id_file
    )
    print(f"Indexed file: {id_file=} in VDB: {id_vdb=}")
else:
    print(f"File: {id_file=} is already indexed in {id_vdb=}")


File: id_file='file-2CpCDMwstr55pejhrrFivr' is already indexed in id_vdb='vs_698f014ef4648191bbcab8191d249673'


# Query

In [12]:
query = "¿A qué distancia de la Tierra está el sistema Althéra?"
res_search = client_openai.vector_stores.search(
    vector_store_id=id_vdb,
    query=query,
    max_num_results=RETRIEVE_K
)

In [13]:
for result in res_search.data:
    print(str(len(result.content[0].text)) + ' of character of content from ' + result.filename + ' with a relevant score of ' + str(result.score))

2738 of character of content from Divulgacion-Planetaria-Althera.pdf with a relevant score of 0.9837203061565574
2628 of character of content from Divulgacion-Planetaria-Althera.pdf with a relevant score of 0.9811685232994241
2411 of character of content from Divulgacion-Planetaria-Althera.pdf with a relevant score of 0.8846177022046398


In [14]:
# —a tan solo 42,7 años luz de la Tierra—
res_search.data[0].content[0].text

'• Telescopio Espacial Nancy Grace Roman para fotometría de gran precisión \r\nen tránsitos. \r\n• Interferometría de radio desde la red Very Long Baseline Array (VLBA) para \r\nafinar la distancia y parámetros orbitales del sistema. Divulgación Planetaria: 2025-12 \r\nPágina 2 | 18\r\nFue el equipo del astrónomo estadounidense Dr. Jonathan Kepler-Saunders quien \r\nconfirmó, mediante el método de velocidad radial ultraestable, la existencia de cinco \r\nplanetas principales y varios cinturones de escombros. \r\n1.3 Descubrimiento revolucionario de la zona habitable circumbinaria \r\nEl hallazgo más impactante llegó en 2034, cuando la misión LUVOIR-B (Large \r\nUV/Optical/IR Surveyor) detectó la firma espectral de vapor de agua, oxígeno \r\nmolecular y metano en la atmósfera de Aurelia III, un planeta ubicado en la zona \r\nhabitable del sistema, orbitando a ambos soles. Este fue el primer caso \r\ndocumentado de un mundo potencialmente habitable en un sistema binario \r\ncercano —a ta

# RAG

In [15]:
query = "¿A qué distancia de la Tierra está el sistema Althéra?"

res_rag = client_openai.responses.create(
    input= query,
    model=LLM,
    tools=[{
        "type": "file_search",
        "vector_store_ids": [id_vdb],
    }]
)

# Extract annotations from the response
annotations = res_rag.output[1].content[0].annotations
    
# Get top-k retrieved filenames
retrieved_files = set([result.filename for result in annotations])

print(f'Files used: {retrieved_files}')
print('Response:')
print(res_rag.output[1].content[0].text) # 0 being the filesearch call

Files used: {'Divulgacion-Planetaria-Althera.pdf'}
Response:
El sistema binario Althéra (HD 4579 AB) está ubicado en el brazo de Orión de la Vía Láctea, a una distancia de aproximadamente 42,7 años luz de la Tierra. Esta distancia relativamente cercana permite una observación detallada del sistema con telescopios avanzados y hace que el sistema sea un candidato ideal para futuras misiones de exploración interestelar.

Esta información aparece claramente en varias secciones del documento, donde se menciona que la proximidad de 42,7 años luz ha sido confirmada mediante mediciones precisas y es fundamental para la estabilidad térmica y habitabilidad de sus planetas, especialmente Aurelia III, que está en la zona habitable circumbinaria del sistema Althéra.


In [16]:
annotations[0]

AnnotationFileCitation(file_id='file-2CpCDMwstr55pejhrrFivr', filename='Divulgacion-Planetaria-Althera.pdf', index=701, type='file_citation')

In [17]:
res_rag.usage.total_tokens

14647

In [19]:
res_rag.output[0]  # ResponseFileSearchToolCall

ResponseFileSearchToolCall(id='fs_0b94f3d4a12a45df00698f04ae9cc881968778254ce35d3ef6', queries=['¿A qué distancia de la Tierra está el sistema Althéra?'], status='completed', type='file_search_call', results=None)

In [20]:
res_rag.output[1]  # ResponseOutputMessage

ResponseOutputMessage(id='msg_0b94f3d4a12a45df00698f04b060148196b2279106f8d38567', content=[ResponseOutputText(annotations=[AnnotationFileCitation(file_id='file-2CpCDMwstr55pejhrrFivr', filename='Divulgacion-Planetaria-Althera.pdf', index=360, type='file_citation')], text='El sistema Althéra (HD 4579 AB) se encuentra a una distancia de aproximadamente 42,7 años luz de la Tierra. Está localizado en el brazo de Orión de la Vía Láctea, cerca de la nebulosa Barnard’s Loop, en la constelación de Orión. Esta información es precisa y está confirmada mediante interferometría de radio y otros métodos de observación astronómica avanzada.', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')

# Structured Outputs

The Augmented LLM

![Augmented LLM](https://www.anthropic.com/_next/image?url=https%3A%2F%2Fwww-cdn.anthropic.com%2Fimages%2F4zrzovbb%2Fwebsite%2Fd3083d3f40bb2b6f477901cc9a240738d3dd1371-2401x1000.png&w=3840&q=75)


Structured Output: Obtain an answer to as a string, but as json format  
Usually, pydantic library is used  


In [19]:
data= """
A LA ATT. DE SEGUROS LLOYD:
EN DON BENITO, A 08 MARZO 2014

YO, CARMEN ESPAÑOLA ESPAÑOLA,
CON DNI 99999999R, QUIERO DARME
DE BAJA DEL SEGURO DE COCHE QUE
TENGO CON USTEDES POR LA VENTA DEL
MISMO.

EL NÚMERO DE PÓLIZA ES h2024038
Y SE CORRESPONDE CON UN OPEL CORSA 1.2L
CON MATRÍCULA 5473 BXM

[signature]

Dta. CARMEN ESPAÑOLA
ESPAÑOLA
"""


In [20]:
response = client_openai.responses.create(
    model="gpt-4.1-2025-04-14",
    input=[
        {
            "role": "system",
            "content": "Eres un analista que se dedica al cribado de solicitudes de seguros\
                Tu misión es leer detenidamente el correo de un asegurado y extraer información clave",
        },
        {
            "role": "user",
            "content": f"Correo de solicitud: {data}"
        },
    ],
    )

print(response.output[0].content[0].text)

**Información clave extraída del correo de solicitud:**

- **Nombre de la asegurada:** Carmen Española Española
- **DNI:** 99999999R
- **Motivo de la solicitud:** Solicitud de baja del seguro de coche por venta del vehículo
- **Número de póliza:** h2024038
- **Vehículo asegurado:** Opel Corsa 1.2L
- **Matrícula del vehículo:** 5473 BXM
- **Fecha de la solicitud:** 08 de marzo de 2014
- **Lugar:** Don Benito

¿Necesitas el resumen en otro formato (tabla, informe, etc.) o algún detalle adicional?


In [33]:
response = client_openai.responses.create(
    model="gpt-4.1-2025-04-14",
    input=[
        {
            "role": "system",
            "content": "Eres un analista que se dedica al cribado de solicitudes de seguros\
                Tu misión es leer detenidamente el correo de un asegurado y extraer información clave en forma de diccionario",
        },
        {
            "role": "user",
            "content": f"Correo de solicitud: {data}"
        },
    ],
    )

response_content = response.output[0].content[0].text
print(f"{type(response_content)=}")
print(f"{response_content=}")

type(response_content)=<class 'str'>
response_content='{\n  "fecha": "08/03/2014",\n  "nombre_asegurado": "Carmen Española Española",\n  "dni": "99999999R",\n  "motivo": "Baja del seguro por venta del vehículo",\n  "número_poliza": "h2024038",\n  "vehículo": {\n    "marca_modelo": "Opel Corsa 1.2L",\n    "matrícula": "5473 BXM"\n  },\n  "aseguradora": "Seguros Lloyd",\n  "localidad": "Don Benito"\n}'


In [23]:
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import Optional



class DatosPoliza(BaseModel):
    fecha: Optional[str] = Field(description="Si existe, la fecha a la que se firma la solicitud, en formato YYYY-MM-DD")
    dni:  Optional[str] = Field(description="Si existe, el DNI o número de pasarporte del solicitante")
    nro_poliza:  Optional[str]  = Field(description="Si existe, el número de póliza sobre el que el solicitante desea realizar una acción")
    marca_model:  Optional[str] = Field(description="Si aparece, la marca y/o el modelo del vehículo asociado a la póliza")
    matricula:  Optional[str] = Field(description="Si aparece, la matrícula vehículo asociado a la póliza")


response = client_openai.responses.parse(
    model="gpt-4.1-2025-04-14",
    input=[
        {
            "role": "system",
            "content": "Eres un analista que se dedica al cribado de solicitudes de seguros\
                Tu misión es leer detenidamente el correo de un asegurado y extraer información clave",
        },
        {
            "role": "user",
            "content": f"Correo de solicitud: {data}"
        },
    ],
    text_format=DatosPoliza,
)

data_extracted = response.output_parsed

In [24]:
data_extracted

DatosPoliza(fecha='2014-03-08', dni='99999999R', nro_poliza='h2024038', marca_model='OPEL CORSA 1.2L', matricula='5473 BXM')

# Nesting

In [25]:
from pydantic import BaseModel, PositiveInt, Field
from typing import List

class Product(BaseModel):
    name: str
    quantity: PositiveInt = Field(description="The number of units of this product.")
    unit_price: float = Field(description="The price per unit of the product.")

class Order(BaseModel):
    order_number: str
    items: List[Product]
    customer_name: str
    total_cost: float

order_text = "Order #12345. I purchased two t-shirts at $25 each, and one hoodie for $50. The total came to $100. My name is Jane Doe."


response = client_openai.responses.parse(
    model="gpt-4.1-2025-04-14",
    input=[
        {
            "role": "system",
            "content": "You analize orders recipients. Your task is to extract key information accuratelly"
        },
        {
            "role": "user",
            "content": f"{order_text}"
        },
    ],
    text_format=Order,
)

data_extracted = response.output_parsed

data_extracted

Order(order_number='12345', items=[Product(name='t-shirt', quantity=2, unit_price=25.0), Product(name='hoodie', quantity=1, unit_price=50.0)], customer_name='Jane Doe', total_cost=100.0)

# Add a predefined set of values

In [26]:
from enum import Enum

class AccionesPoliza(str, Enum):
    BAJA = "BAJA"
    REVISION = "REVISION"
    ANULACION = "ANULACION"
    
    
class DatosPoliza(BaseModel):
    fecha: Optional[str] = Field(description="Si existe, la fecha a la que se firma la solicitud")
    dni:  Optional[str] = Field(description="Si existe, el DNI o número de pasarporte del solicitante")
    accion:  Optional[AccionesPoliza] = Field(description="Si se expresa claramente, la acción que se solicita.")
    nro_poliza:  Optional[str]  = Field(description="Si existe, el número de póliza sobre el que el solicitante desea realizar una acción")
    marca_model:  Optional[str] = Field(description="Si aparece, la marca y/o el modelo del vehículo asociado a la póliza")
    matricula:  Optional[str] = Field(description="Si aparece, la matrícula vehículo asociado a la póliza")



response = client_openai.responses.parse(
    model="gpt-4.1-2025-04-14",
    input=[
        {
            "role": "system",
            "content": "Eres un analista que se dedica al cribado de solicitudes de seguros\
                Tu misión es leer detenidamente el correo de un asegurado y extraer información clave",
        },
        {
            "role": "user",
            "content": f"Correo de solicitud: {data}"
        },
    ],
    text_format=DatosPoliza,
)

data_extracted = response.output_parsed

data_extracted

DatosPoliza(fecha='08 MARZO 2014', dni='99999999R', accion=<AccionesPoliza.BAJA: 'BAJA'>, nro_poliza='h2024038', marca_model='OPEL CORSA 1.2L', matricula='5473 BXM')

# Other tools

[LangExtract](https://github.com/google/langextract) released by Google in July 30th, is a open source project for large scale, model agnostic (but Gemini first) and example guided information extraction